In [6]:
import torch.nn as nn
import torch
import math

batch_size = 1
image_size = 14
dim_in = 3
emb_dim = 16
c_group = 1
kernel_size = 3
stride = 2
c_group = 1
gamma = 0.5
beta = 0.2

running_mean = torch.zeros(emb_dim)
running_var = torch.zeros(emb_dim)
for i in range(emb_dim):
    running_mean[i] = 8
    running_var[i] = 21.5

image = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                image[b, c, i, j] = (b+c+i+j) % 256

kernel = torch.zeros(emb_dim, dim_in, kernel_size, kernel_size)
kernel[:, :, :, :] = 0.1

c_group = math.gcd(dim_in, emb_dim) if c_group == 1 else c_group
padding = math.ceil(((kernel_size - 1) * 1 + 1 - stride) / 2)
print('padding: ', padding)
conv = nn.Conv2d(dim_in, emb_dim, kernel_size, stride, padding, dilation=1, groups=c_group, bias=False)
conv.weight.data = kernel
image = conv(image)
print('after conv')
print(image[0, 0, :, :])

norm = nn.BatchNorm2d(emb_dim, momentum=0)
norm.running_mean = running_mean
norm.running_var = running_var
norm.weight.data.fill_(gamma)
norm.bias.data.fill_(beta)
norm.eval()
result = norm(image)

print(result.shape)
print(result[0, 0, :, :])

padding:  1
after conv
tensor([[ 2.4000,  6.3000,  9.9000, 13.5000, 17.1000, 20.7000, 24.3000],
        [ 6.3000, 13.5000, 18.9000, 24.3000, 29.7000, 35.1000, 40.5000],
        [ 9.9000, 18.9000, 24.3000, 29.7000, 35.1000, 40.5000, 45.9000],
        [13.5000, 24.3000, 29.7000, 35.1000, 40.5000, 45.9000, 51.3000],
        [17.1000, 29.7000, 35.1000, 40.5000, 45.9000, 51.3000, 56.7000],
        [20.7000, 35.1000, 40.5000, 45.9000, 51.3000, 56.7000, 62.1000],
        [24.3000, 40.5000, 45.9000, 51.3000, 56.7000, 62.1000, 67.5000]],
       grad_fn=<SliceBackward0>)
torch.Size([1, 16, 7, 7])
tensor([[-0.4039,  0.0167,  0.4049,  0.7931,  1.1813,  1.5695,  1.9577],
        [ 0.0167,  0.7931,  1.3754,  1.9577,  2.5400,  3.1223,  3.7046],
        [ 0.4049,  1.3754,  1.9577,  2.5400,  3.1223,  3.7046,  4.2869],
        [ 0.7931,  1.9577,  2.5400,  3.1223,  3.7046,  4.2869,  4.8692],
        [ 1.1813,  2.5400,  3.1223,  3.7046,  4.2869,  4.8692,  5.4515],
        [ 1.5695,  3.1223,  3.7046,  4.28